In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata

# File names
known_file = "output_coordinates_figure_3_magnetic_data_HR_with_IGRF_filtered_100.txt"
coordinates_file = "coordinates_figure_3_magnetic_data_HR_with_IGRF.txt"
output_file = "interpolated_results_coordinates_figure_3_magnetic_data_HR_with_IGRF.txt"

# Load known data, ensuring correct columns are read
known_data = pd.read_csv(known_file, delim_whitespace=True)

# Check column names
print("Known Data Columns:", known_data.columns)

# Drop any unexpected extra columns (e.g., 'year' if present)
if 'year' in known_data.columns:
    known_data = known_data.drop(columns=['year'])

# Load target coordinates
target_coords = pd.read_csv(coordinates_file, delimiter=",", header=0, names=["lat", "lon"])

# Ensure correct column selection for interpolation
points = known_data.iloc[:, [0, 1]].values  # Use only (glat, glon)
values1 = known_data.iloc[:, 2].values  # Ensure this is mlat
values2 = known_data.iloc[:, 3].values  # Ensure this is mlon

# Target coordinates for interpolation
new_points = target_coords[["lat", "lon"]].values

# Perform interpolation
interpolated_value1 = griddata(points, values1, new_points, method="linear")
interpolated_value2 = griddata(points, values2, new_points, method="linear")

# Handle NaN values by using nearest neighbor fallback
nan_mask1 = np.isnan(interpolated_value1)
nan_mask2 = np.isnan(interpolated_value2)

if np.any(nan_mask1) or np.any(nan_mask2):
    print("Warning: Some points were outside interpolation range. Using nearest-neighbor fallback.")
    interpolated_value1[nan_mask1] = griddata(points, values1, new_points, method="nearest")[nan_mask1]
    interpolated_value2[nan_mask2] = griddata(points, values2, new_points, method="nearest")[nan_mask2]

# Store results in DataFrame
target_coords["interpolated_mlat"] = interpolated_value1
target_coords["interpolated_mlon"] = interpolated_value2

# Save to a file
target_coords.to_csv(output_file, sep=" ", index=False, header=True)

print(f"Interpolated values saved to {output_file}")
